In [3]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import os

class COVIDClassificationApp:
    def __init__(self, master):
        self.master = master
        master.title("COVID-19 X-ray Classification")
        master.geometry("800x600")
        master.configure(bg='#5af6f1')

        # Load pre-trained TensorFlow model
        self.save_path = r'C:\Users\Sajad\MVProject\cnn_model2.h5'  # Update this with your actual model path
        self.model = load_model(self.save_path)

        # Image preprocessing transformations
        self.input_shape = (224, 224)  # Adjust based on your model's input size

        # Create UI components
        self.create_widgets()

    def create_widgets(self):
        # Image Display Frame
        self.image_frame = tk.Frame(self.master, width=500, height=400, bg='white', relief=tk.RAISED, borderwidth=2)
        self.image_frame.pack(pady=20)
        self.image_frame.pack_propagate(False)

        self.image_label = tk.Label(self.image_frame, text="No Image Selected", bg='white')
        self.image_label.pack(expand=True)

        # Button Frame
        button_frame = tk.Frame(self.master, bg='#5af6f1')
        button_frame.pack(pady=10)

        # Select Image Button
        select_button = tk.Button(button_frame, text="Select Image", command=self.select_image, 
                                  bg='#fc1b55', fg='white', font=('Arial', 12))
        select_button.pack(side=tk.LEFT, padx=10)

        # Classify Button
        classify_button = tk.Button(button_frame, text="Classify Image", command=self.classify_image, 
                                    bg='#2e33da', fg='white', font=('Arial', 12))
        classify_button.pack(side=tk.LEFT, padx=10)

        # Result Frame
        self.result_frame = tk.Frame(self.master, width=600, height=100, bg='#00c9cc', relief=tk.SUNKEN, borderwidth=2)
        self.result_frame.pack(pady=10)
        self.result_frame.pack_propagate(False)

        self.result_label = tk.Label(self.result_frame, text="Classification Result", 
                                     bg='#00c9cc',fg='white', font=('Arial', 14))
        self.result_label.pack(expand=True)

        # Class labels
        self.class_labels = ["Normal", "COVID-19"]

        # Store selected image path
        self.selected_image_path = None

    def select_image(self):
        # Open file dialog to select image
        file_path = filedialog.askopenfilename(
            title="Select X-ray Image",
            filetypes=[("Image files", "*.png *.jpg *.jpeg *.bmp *.tiff")]
        )
        
        if file_path:
            self.selected_image_path = file_path
            # Display selected image
            image = Image.open(file_path)
            image.thumbnail((500, 400))  # Resize for display
            photo = ImageTk.PhotoImage(image)
            
            self.image_label.configure(image=photo)
            self.image_label.image = photo  # Keep a reference
            
            # Reset result label
            self.result_label.configure(text="Image Selected. Ready to Classify.")

    def classify_image(self):
        if not self.selected_image_path:
            messagebox.showwarning("Warning", "Please select an image first!")
            return

        try:
            # Open and preprocess image
            image = Image.open(self.selected_image_path).convert('RGB')
            image = image.resize(self.input_shape)  # Resize to match model input
            
            # Convert to numpy array and normalize
            img_array = img_to_array(image)
            img_array = img_array / 255.0  # Normalize to [0,1]
            img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

            # Perform classification
            prediction = self.model.predict(img_array)
            predicted_class = np.argmax(prediction, axis=1)[0]
            confidence = np.max(prediction) * 100

            # Update result label
            result_text = f"Prediction: {self.class_labels[predicted_class]}\n" \
                          f"Confidence: {confidence:.2f}%"
            self.result_label.configure(text=result_text, fg='green')

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

def main():
    root = tk.Tk()
    app = COVIDClassificationApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()


1/1 [==============================] - 1s 755ms/step
